In [ ]:
model_name = 'nn_sequential'
model_variant = 'base'

In [ ]:
import pandas as pd

df = pd.read_csv('spotify_dataset.csv')
df = df.drop(columns=['Song ID', 'Artist', 'Number of Times Charted',
 'Week of Highest Charting', 'Song Name', 'Streams', 'Release Date',
  'Weeks Charted', 'Popularity', 'Index'])
df = df.replace(' ','0')

print('Shape of dataframe is:', df.shape)

df.head(5)

In [ ]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
df['Chord'] = enc.fit_transform(df.Chord.values)
df.head()

In [ ]:
df['Artist Followers'] = df['Artist Followers'].values.astype(int)
df['Danceability'] = df['Danceability'].values.astype(float)
df['Energy'] = df['Energy'].values.astype(float)
df['Loudness'] = df['Loudness'].values.astype(float)
df['Speechiness'] = df['Speechiness'].values.astype(float)
df['Acousticness'] = df['Acousticness'].values.astype(float)
df['Liveness'] = df['Liveness'].values.astype(float)
df['Tempo'] = df['Tempo'].values.astype(float)
df['Duration (ms)'] = df['Duration (ms)'].values.astype(int)
df['Valence'] = df['Valence'].values.astype(float)

y = df['Highest Charting Position'].values

split = round(len(df)*0.6)
X_train = df[:split]
y_train = y[:split]
X_test = df[split:]
y_test = y[split:]

print('Shape of X_train is:', X_train.shape)
print('Shape of y_train is:', y_train.shape)
print('Shape of X_test is:', X_test.shape)
print('Shape of y_test is:', y_test.shape)

In [ ]:
# SETTING UP KERAS CALLBACKS

import os
import time
import tensorflow as tf
from keras import callbacks

# Borrowed from: https://www.geeksforgeeks.org/choose-optimal-number-of-epochs-to-train-a-neural-network-in-keras/
earlystopping = callbacks.EarlyStopping(monitor ="val_loss", 
                                        mode ="min", patience = 10, 
                                        restore_best_weights = True)

def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        exp = lr0 * 0.1**(epoch / s)
        tf.summary.scalar('learning rate', data=exp, step=epoch)
        return exp
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(lr0=0.01, s=10)
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(exponential_decay_fn)

root_logdir = os.path.join(os.curdir, "./logged_models/" + model_name + '/' + model_variant)
def get_run_logdir():
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_log_dir = get_run_logdir()
file_writer = tf.summary.create_file_writer(run_log_dir + "/metrics")
file_writer.set_as_default()

tensorboard_cb = tf.keras.callbacks.TensorBoard(run_log_dir)

my_callbacks = [earlystopping]

In [ ]:
from tensorflow.keras import layers

model = tf.keras.Sequential([
        layers.Dense(2, activation="relu"),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),
])

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train['Artist Followers'], y_train, epochs=50, callbacks=my_callbacks, validation_split=0.2)

model.summary()